In [31]:
import stim
import numpy as np
import qiskit
import itertools
import nbimporter

from NonCliff_1Meas_V4 import *
from stab_rank_approx import *

from qiskit.quantum_info import Pauli, PauliList, StabilizerState
from typing import List, Tuple

In [45]:
def iterated_optimal_nearest_neighbor_V2(num_qubits:int, eig_vec:np.array, init_stab_state: np.array, used_states: List) -> np.array:
    """ 
    Iteratively compute the optimal nearest neighbor whose inner product with 'eig_vec' is maximal

    Parameters:
    -----------
    num_qubits - Number of qubits
    eig_vec - Eigenvector we wish to approximate
    init_stab_state - Initial stabilizer state where we start our search from 

    Returns:
    --------
    Optimal single state that best approximates 'eig_vec'
    """
    num_iterations = 2 ** num_qubits
    curr_stab_state = init_stab_state
    max_inner_prod = 0
    curr_best_stab_state = None
    for _ in range(num_iterations):
        curr_neighbors = find_nearest_neighbors_general(num_qubits, curr_stab_state)
        optimal_neighbor = curr_neighbors[np.argmax([np.vdot(neighbor, eig_vec) for neighbor in curr_neighbors])]
        curr_stab_state = optimal_neighbor 
        curr_inner_prod = np.vdot(curr_stab_state, eig_vec)
        if (curr_inner_prod > max_inner_prod):
            #if (np.any(np.array_equal(curr_stab_state, s) for s in used_states)):
            #    continue
            max_inner_prod = curr_inner_prod 
            curr_best_stab_state = curr_stab_state

    return curr_best_stab_state, max_inner_prod

In [48]:
def one_state_stab_approx_V2(num_qubits:int, eig_vec: np.array, used_states: List) -> np.array:
    """ 
    Find best (?) single stabilizer state approximation for eigen vector 

    Parameters:
    -----------
    num_qubits - Number of qubits
    eig_vec - Eigenvector we wish to approximate

    Returns:
    --------
    Best (?) single stabilizer state that approximates eigenvector
    """

    basis_vecs = comp_base_vecs(num_qubits) # Basis vectors given number of qubits
    init_state = eig_vec.copy() # Make a copy of the original eigenvector
    #final_approx_state = np.zeros((2**num_qubits,), dtype=complex) # Where result for best stab state approx is stored
    
    # Find basis state with maximum coefficient
    max_coeff = state_max_coeff(eig_vec, basis_vecs)
    init_stab_state = basis_vecs[max_coeff]

    # Compute optimal nearest stab state in iterated fashion
    final_approx_state, max_inner_prod = iterated_optimal_nearest_neighbor_V2(num_qubits, eig_vec, init_stab_state, used_states)

    return final_approx_state, max_inner_prod

In [93]:
def check_orthogonality(used_states: List, curr_stab: np.array):
    """
    Checks that all the stabilizer states that are being used for decomposition are indeed orthogonal

    Params:
    -------
    used_states - stabilizer states used so far 
    curr_stab - Next stabilizer state to be used for decomposition
    """ 
    for elem in used_states:
        if (np.vdot(curr_stab, elem) != 0):
            return False 
    return True 

def stab_rank_approx_four_V2(num_qubits: int, eig_vec: np.array) -> Tuple:
    """ 
    Approximate the stabilizer rank (using approach discussed)

    Parameters:
    -----------
    num_qubits - Number of qubits
    eig_vec - Eigenvector for which we'd like to evaluate stabilizer rank

    Returns:
    --------
    Approximate stabilizer rank for eigenvector
    """
    approx_stab_rank = 0 
    approx_vec = np.zeros(len(min_eigvec), dtype=complex) # Vector we wish to construct to approximate 'eig_vec'
    prec_param = 0.04 # Precision parameter to see how close dot prod of 'approx_vec' and 'eig_vec' is
    used_states = []
    norm_factor = 0
    is_approx = False # While loop termination condition

    copy_vec = eig_vec.copy()
    while (is_approx == False):
        # Calculate nearest stabilizer state
        s, s_inner_prod = one_state_stab_approx_V2(num_qubits, eig_vec, used_states)
        print("Is orthogonal?: " + str(check_orthogonality(used_states, s)))
        used_states.append(s)
        print("Stab: " + str(list(s)))

        # Project 'eig_vec' on s
        s_norm = np.linalg.norm(s)
        s_coeff = np.vdot(s, eig_vec)
        if (norm_factor == 0):
            proj_vec = (s_coeff/s_norm**2) * s
        else:
            proj_vec = norm_factor * (s_coeff/s_norm**2) * s

        # Subtract projection of 'eig_vec' on s from 'eig_vec'
        unnorm_eig_vec = eig_vec - proj_vec 
        norm_factor = np.linalg.norm(unnorm_eig_vec)
        eig_vec = unnorm_eig_vec / norm_factor

        # Add projection to approximation and update stab rank
        approx_vec += proj_vec
        approx_stab_rank += 1

        # Check closeness of approximation to original 'eig_vec'
        print(np.vdot(approx_vec / np.linalg.norm(approx_vec), copy_vec))
        if (np.abs(1 - np.vdot(approx_vec / np.linalg.norm(approx_vec), copy_vec)) <= prec_param):
            is_approx = True 
        #print(np.linalg.norm((approx_vec / np.linalg.norm(approx_vec)) - copy_vec))
        #if (np.linalg.norm((approx_vec / np.linalg.norm(approx_vec)) - copy_vec) <= prec_param):
        #    is_approx = True

        if (approx_stab_rank == 2 ** num_qubits):
            break

        print(list(approx_vec))
        
    approx_vec = approx_vec / np.linalg.norm(approx_vec)
    
    return approx_stab_rank, approx_vec


In [100]:
num_wit_qubits = 25 # variable 'n' in paper
# For now, what the witness qubits are initialized to be doesn't matter (I THINK)
# as we are trying to minimize Val, given some optimal input states.
wit_qubits = np.zeros(num_wit_qubits, dtype=complex)

num_anc_qubits = 0 # variable 'm' in paper
anc_qubits = np.zeros(num_anc_qubits, dtype=complex)

num_t_gates = 4 # variable 't' in paper

num_meas_qubits = 1 # variable 'k' in paper
total_qubits = num_wit_qubits + num_anc_qubits

num_layers = 20

assert(num_layers > 0)

In [104]:
p_sum, _ = sim_circ(total_qubits, num_layers, num_t_gates)
succinct_p_sum = p_sum.succinct_p_sum(total_qubits)
min_eigvec = succinct_p_sum.compute_min_eig_eigvec(len(p_sum.get_bases()))[1]
new_total_qubits = len(succinct_p_sum.get_sum()[0].get_pauli_str(succinct_p_sum.get_sum()[0].get_pauli()))
print(succinct_p_sum)
print(list(min_eigvec))

[(0.3535533905932737+0j) ZIIII] + [(-0.24999999999999992+0j) YZIII] + [(-0.3535533905932737+0j) YIIII] + [(-0.24999999999999992+0j) ZZIII] + [(-0.24999999999999992+0j) XYXZI] + [(0.3535533905932737+0j) IXXZI] + [(0.24999999999999992-0j) IYXZI] + [(-0.3535533905932737+0j) XXXZI] + [(0.24999999999999992-0j) YXIXI] + [(0.24999999999999992-0j) ZXIXI] + [(0.24999999999999992+0j) XIXYI] + [(0.24999999999999992-0j) IIXYI]
[(0.0008877243349877717-0.00167768453404448j), (0.07400627418029808+0.14552496575670004j), (0.019194072255425826-0.09903155898480699j), (-0.08838005301141624+0.14085121016867566j), (-0.02504165419810756+0.049091834398506506j), (-0.021748208063286076+0.03389188035142385j), (0.12752544834671722-0.09365259806687298j), (-0.3096813207551338-0.04859375680778211j), (-0.025622849693075826+0.02818761526881014j), (0.08219523429853982-0.055140534290931006j), (0.036207595350246716-0.026115730076360106j), (-0.08919692926080372-0.10159695913301012j), (-0.08325854227331349+0.01409936495692

In [105]:
approx_stab_rank, approx_vec = stab_rank_approx_four_V2(new_total_qubits, min_eigvec)
print(approx_stab_rank, approx_vec)

Is orthogonal?: True
Stab: [0j, 0.4999999999999999j, 0j, 0j, 0j, 0j, 0j, (-0.4999999999999999+0j), 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, (0.4999999999999999+0j), 0j, 0.4999999999999999j, 0j, 0j]
(0.628042049758951+0j)
[0j, (0.01955858540488468+0.3134113364307615j), 0j, 0j, 0j, 0j, 0j, (-0.3134113364307615+0.01955858540488468j), 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, (0.3134113364307615-0.01955858540488468j), 0j, (0.01955858540488468+0.3134113364307615j), 0j, 0j]
Is orthogonal?: False
Stab: [0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, (0.24999999999999992-0.24999999999999992j), (-0.24999999999999992-0.24999999999999992j), 0j, 0j, (0.24999999999999992+0.24999999999999992j), (0.24999999999999992-0.24999999999999992j), 0j, 0j, (-0.24999999999999992+0.24999999999999992j), (0.24999999999999992+0.24999999999999992j), 0j, 0j, (0.24999999999999992+0.24999999999999992j), (0.24999999999999992-0.

KeyboardInterrupt: 

In [87]:
print(min_eigvec)
print(np.vdot(min_eigvec, approx_vec))

[ 0.00231543-0.j  0.21659858-0.j -0.00231543-0.j  0.21659858-0.j
 -0.31484011-0.j  0.04351239-0.j  0.31484011-0.j  0.04351239-0.j
 -0.43281768-0.j  0.20946913-0.j  0.43281768+0.j  0.20946913-0.j
  0.22040237-0.j  0.26884272-0.j -0.22040237-0.j  0.26884272-0.j]
(0.9721400017546745+0j)
